In [20]:
from openai import OpenAI
import toml
from ollama import chat
from ollama import ChatResponse
from ollama import Client
from langchain_core.prompts import PromptTemplate


In [9]:

# Load a TOML file
with open('../secrets.toml', 'r') as f:
    openai_key = toml.load(f).get('api_keys').get('openai')

assert openai_key, "OpenAI API key not found in secrets.toml"


In [56]:
class LLMModel:
    def __init__(self, model_name: str, run_local: bool = True, system_prompt: str = "you are a helpful assistant"):
        self.run_local = run_local
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": self.system_prompt}]
        
        if self.run_local:
            self.client = Client(
                host='http://localhost:11434',
                # headers={'x-some-header': 'some-value'}
                )

    def add_message(self, role: str, content: str) -> bool:
        if role not in ["system", "user", "assistant"]:
            raise ValueError("Role must be 'system', 'user', or 'assistant'")
        self.messages.append({"role": role, "content": content})
        return True

    def clear_messages(self) -> bool:
        self.messages = []
        return True

    def local_generate(self, prompt: str, max_tokens: int = 1500, **kwargs) -> str:
        self.messages.append({"role": "user", "content": prompt})
        response = self.client.chat(model=self.model_name, messages=self.messages, **kwargs)
        self.messages.append({"role": "assistant", "content": response.message.content.strip()})
        return response.message.content.strip()
    
    # question: how do I set max token in ollama python api?
    


In [57]:
model = LLMModel(model_name="llama3.2:latest", run_local=True)

In [12]:
# model.local_generate("What is the capital of India?")

In [13]:
# model.local_generate("What is the weather like there?")

In [14]:
# model.local_generate("What is the best month to visit?")

In [15]:
# model.local_generate("What is the best month to visit? Just give me the month name, no other text.")

In [16]:
# model.messages

In [ ]:
# model.clear_messages()

True

In [58]:
topic = "The Emergency in India (1975-1977)"

In [68]:
generate_subtopics = PromptTemplate.from_template(
    """Generate a list of 5 subheadings for a highschool essay on {topic}. Provide output in json format as follows:
    {{'<title>':'<description>',
    '<title>':'<description>',...}}
    """
)

In [69]:
subtopics_prompt = generate_subtopics.format(topic=topic)
subtopics_prompt

"Generate a list of 5 subheadings for a highschool essay on The Emergency in India (1975-1977). Provide output in json format as follows:\n    {'<title>':'<description>',\n    '<title>':'<description>',...}\n    "

In [70]:
model.local_generate(subtopics_prompt, format='json')

'{"Declarations of Emergency: What Led to the 1975 Crisis?":"An analysis of Indira Gandhi\'s motivations and decisions that led to the imposition of the Emergency",\n"The Impact on India\'s Democracy: A Critical Examination of the Emergency Period":"A discussion on how the Emergency affected the country\'s constitutional framework and democratic institutions",\n"Rise of Opponents: The Emergence of Opposition Movements During the Emergency":"An exploration of the various opposition movements that emerged during the Emergency period, including the Congress (R) Party and the Janata Party",\n"Economic Reforms Under Emergency Rule":"An examination of the economic policies implemented by the government during the Emergency, including price controls, nationalization, and export restrictions",\n"Legacy of the Emergency: The Road to Restitution of Democracy in India":"A consideration of the lasting impact of the Emergency on Indian politics and society, and the steps taken to restore democracy 